# ボートレーサーごとの統計量をdfに追加

## 1. 統計解析用のdfをロード
- `race_df`に格納
- 2_データ前処理.ipnbを参照

In [1]:
import pandas as pd
race_df = pd.read_pickle('../../data/arrangedData/data_formatted_for_statistic.pkl').reset_index(drop=True)
# 中身を確認
race_df.head()

,date,venue,raceNumber,枠,boatNo,motorNo,num_false_start,num_late_start,racer_class,racer_id,...,water_temperature,wave_height,weather,weight,wind_speed,タイム,着順,weather_曇り,weather_雨,weather_雪
0,2021-02-01,津,1.0,1,38,20,0,0,1,4460,...,6.0,1,曇り,52.2,1,"1'46""6",1,1,0,0
1,2021-02-01,津,1.0,2,28,26,0,0,3,5112,...,6.0,1,曇り,53.1,1,"1'49""8",2,1,0,0
2,2021-02-01,津,1.0,3,69,51,0,0,3,4453,...,6.0,1,曇り,53.2,1,"1'52""1",3,1,0,0
3,2021-02-01,津,1.0,4,36,15,0,0,3,3842,...,6.0,1,曇り,52.7,1,"1'53""2",4,1,0,0
4,2021-02-01,津,1.0,5,13,36,0,0,4,4916,...,6.0,1,曇り,52.9,1,,5,1,0,0


## 2. これまでのレース結果から統計量を算出
- 各選手が、1枠から1着になった確率と2-6枠から3着以内に入った確率を算出し、`rank_ratio`列に格納
- 選手ごと、枠ごと、日付ごとに、その前日までのレースを対象に統計量を算出

In [2]:
from tqdm.notebook import tqdm

def calc_win_ratio(data):
    # 1枠については1着率を算出
    total_count = data.count()
    win_count = (data == 1).sum()
    win_ratio = win_count / total_count
    
    return win_ratio

def calc_in_3_ratio(data):
    # 2枠から6枠については3着以内だった率を算出
    total_count = data.count()
    in_3_count = (data < 4).sum()
    in_3_ratio = in_3_count / total_count
    return in_3_ratio


date_list = race_df["date"].unique()

stocastic_df_list = []

for date in tqdm(date_list):
    race_df_before_the_date = race_df[race_df["date"] < date]

    # data frameを1枠とそれ以外に分ける
    race_df_before_the_date_frame_1 = race_df_before_the_date[race_df_before_the_date["枠"]==1]
    race_df_before_the_date_frame_2_6 = race_df_before_the_date[~(race_df_before_the_date["枠"]==1)]

    # 1枠を集めたdfに対して、選手ごとに一着率を算出
    stocastic_df_1 = race_df_before_the_date_frame_1.groupby("racer_id")["着順"].apply(calc_win_ratio).reset_index()
    stocastic_df_1["枠"] = 1

    # 2-6枠の選手に対しては3着以内に入った率を算出
    stocastoc_df_2_6 = race_df_before_the_date_frame_2_6.groupby(["racer_id", "枠"])["着順"].apply(calc_win_ratio).reset_index()

    # 1枠に対する統計量のdfと2枠以降に対する統計量のdfをconcat
    stocastic_df_date = pd.concat([stocastic_df_1, stocastoc_df_2_6])

    # dfにdate列を追加
    stocastic_df_date["date"] = date

    stocastic_df_list.append(stocastic_df_date)

stocastic_df = pd.concat(stocastic_df_list)
# concatした際にindexgが列になって加わるので消しておく
stocastic_df.drop(columns="index", inplace=True)

# 算出した統計量の列名を変更：rank ratio: 1着についてはその日までの過去レースで1着になった確率、2-6枠については3着以内に入った確率
stocastic_df.rename(columns={'着順': 'rank_ratio'}, inplace=True)

stocastic_df

  0%|          | 0/52 [00:00<?, ?it/s]

,racer_id,rank_ratio,枠,date
0,3044.0,0.0,1.0,2021-02-02
1,3158.0,0.0,1.0,2021-02-02
2,3187.0,1.0,1.0,2021-02-02
3,3220.0,0.0,1.0,2021-02-02
4,3226.0,0.0,1.0,2021-02-02
...,...,...,...,...
7372,5174.0,0.0,5.0,2021-03-25
7373,5174.0,0.0,6.0,2021-03-25
7374,5175.0,0.0,4.0,2021-03-25
7375,5175.0,0.0,5.0,2021-03-25


## 3. `race_df`に2で算出した統計量をマージ

In [3]:
merge_key_list = ["date", "枠", "racer_id"]
race_df_w_stocastic = pd.merge(race_df, stocastic_df,
                               left_on=merge_key_list, right_on=merge_key_list,
                               how="left"
                              )
race_df_w_stocastic

,date,venue,raceNumber,枠,boatNo,motorNo,num_false_start,num_late_start,racer_class,racer_id,...,wave_height,weather,weight,wind_speed,タイム,着順,weather_曇り,weather_雨,weather_雪,rank_ratio
0,2021-02-01,津,1.0,1,38,20,0,0,1,4460,...,1,曇り,52.2,1,"1'46""6",1,1,0,0,NaN
1,2021-02-01,津,1.0,2,28,26,0,0,3,5112,...,1,曇り,53.1,1,"1'49""8",2,1,0,0,NaN
2,2021-02-01,津,1.0,3,69,51,0,0,3,4453,...,1,曇り,53.2,1,"1'52""1",3,1,0,0,NaN
3,2021-02-01,津,1.0,4,36,15,0,0,3,3842,...,1,曇り,52.7,1,"1'53""2",4,1,0,0,NaN
4,2021-02-01,津,1.0,5,13,36,0,0,4,4916,...,1,曇り,52.9,1,,5,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43519,2021-03-25,鳴 門,12.0,2,76,66,0,0,3,3620,...,2,晴,54.4,2,,6,0,0,0,0.142857
43520,2021-03-25,鳴 門,12.0,3,77,55,0,0,4,4571,...,2,晴,52.0,2,"1'48""2",1,0,0,0,0.166667
43521,2021-03-25,鳴 門,12.0,4,81,28,0,0,1,4174,...,2,晴,51.5,2,"1'49""3",2,0,0,0,0.142857
43522,2021-03-25,鳴 門,12.0,5,34,58,0,0,2,3568,...,2,晴,52.2,2,"1'51""7",4,0,0,0,0.000000


## 4. 3.で作った統計量入りのdfをpickleファイルにして保存

In [4]:
race_df_w_stocastic.to_pickle('../../data/arrangedData/race_df_w_stocastic.pkl')